In [1]:
import pandas as pd
import requests
import io
import re
import asyncio
from functools import partial
from bs4 import BeautifulSoup
from IPython.core.debugger import set_trace

In [2]:
def etf_alloc(*etfs):
    
    def _name(df):
        name = df.index.name
        if name.lower() != 'region':
            return name
        elif df.index.str.contains('america|asia|europe|africa|middle', case=False).any():
            return name
        else:
            return 'Market Tier'
    
    def _df(df, etf):
        return pd.DataFrame({etf.upper():df.dropna().Percentage.str.rstrip('%').astype('float')})

    
    async def get_tables(etf):
        url = 'https://etfdb.com/etf/' + etf
        read_html_partial = partial(pd.read_html, attrs={'class':'chart base-table'}, index_col=0, flavor=['lxml', 'bs4'])
        tables = await loop.run_in_executor(None, read_html_partial, url)
        return {_name(df):_df(df, etf) for df in tables}


    async def main():
        fts = [asyncio.ensure_future(get_tables(etf)) for etf in etfs]
        return await asyncio.gather(*fts)
    
    asyncio.set_event_loop(asyncio.new_event_loop())
    loop = asyncio.get_event_loop()
    
    # 다음 코드를 주피터에서 돌리려면, tornado를 downgrade 해야함
    # pip install tornado==4.5.3
    res = loop.run_until_complete(main())
    loop.close()
    
    etfs = {
        k:pd.concat([dic[k] for dic in res], axis=1, sort='False').fillna(0) for k in res[0]
    }
    
    return pd.concat(etfs, axis=0)

In [3]:
%%time
etfs = etf_alloc('SPY','ACWI')#,'MTUM','VLUE','QUAL','XLB','XLY','XLP','XLE','XLF','XLV','XLI','IYR'); etfs

Wall time: 3.03 s


In [9]:
etfs

SPY   ACWI    MTUM   VLUE   QUAL  \
Asset       Common equity              99.92  98.06   99.69  99.76  99.69   
            ETF Cash Component          0.08   1.28    0.31   0.24   0.31   
            Preferred stock             0.00   0.67    0.00   0.00   0.00   
Country     Argentina                   0.00   0.03    0.00   0.00   0.00   
            Australia                   0.00   2.13    0.00   0.00   0.00   
            Austria                     0.00   0.08    0.00   0.00   0.00   
            Belgium                     0.00   0.31    0.00   0.00   0.00   
            Brazil                      0.00   0.82    0.00   0.00   0.00   
            Canada                      0.00   3.06    0.00   0.00   0.00   
            Chile                       0.00   0.11    0.00   0.00   0.00   
            China                       0.00   3.33    0.00   0.00   0.00   
            Colombia                    0.00   0.05    0.00   0.00   0.00   
            Czech Republic              0.00   0.02    0.00   0.00   0.00   
            Denmark                     0.00   0.57    0.00   0.00   0.00   
            Egypt                       0.00   0.01    0.00   0.00   0.00   
            Finland                     0.00   0.42    0.00   0.00   0.00   
            France                      0.00   3.02    0.00   0.07   0.00   
            Germany                     0.00   2.72    0.00   0.00   0.00   
            Greece                      0.00   0.02    0.00   0.00   0.00   
            Hong Kong                   0.00   1.04    0.00   0.00   0.00   
            Hungary                     0.00   0.05    0.00   0.00   0.00   
            India                       0.00   0.95    0.00   0.00   0.00   
            Indonesia                   0.00   0.20    0.00   0.00   0.00   
            Ireland                     0.92   0.70    0.00   2.44   1.50   
            Israel                      0.00   0.16    0.00   0.00   0.00   
            Italy                       0.00   0.67    0.00   0.00   0.00   
            Japan                       0.00   7.11    0.00   0.14   0.00   
            Liechtenstein               0.00   0.01    0.00   0.00   0.00   
            Luxembourg                  0.00   0.08    0.00   0.00   0.00   
            Malaysia                    0.00   0.25    0.00   0.00   0.00   
...                                      ...    ...     ...    ...    ...   
Market Cap  Large cap                  91.60  89.75   93.50  79.49  89.58   
            Micro cap                   0.00   0.01    0.00   0.00   0.00   
            Mid cap                     8.21   8.59    6.00  19.46   9.91   
            Small cap                   0.11   0.37    0.20   0.81   0.21   
            Unknown                     0.00   0.00    0.00   0.00   0.00   
Market Tier Developed Markets (ex-US)   1.88  36.23    0.00   3.70   3.22   
            Emerging Markets            0.00   9.15    0.00   0.00   0.00   
            Frontier Markets            0.00   0.03    0.00   0.00   0.00   
            Not-designated              0.00   0.01    0.00   0.00   0.00   
            United States              98.12  54.57  100.00  96.30  96.78   
Region      Africa                      0.00   0.76    0.00   0.00   0.00   
            Asia                        0.00   7.80    0.00   0.00   0.00   
            Asia-Pacific                0.00  10.85    0.00   0.34   0.00   
            Europe                      1.88  21.27    0.00   3.36   3.22   
            Latin America               0.00   1.31    0.00   0.00   0.00   
            Middle East                 0.00   0.37    0.00   0.00   0.00   
            North America              98.12  57.63  100.00  96.30  96.78   
Sector      Basic Materials             2.29   4.30    0.14   2.70   2.62   
            Communications             10.18   8.91   11.56  10.21  10.28   
            Consumer, Cyclical          9.67   9.97   13.49   9.82   9.03   
            Consumer, Non-Cyclical   